# Taller: Clasificación con Diabetes Health Indicators

Fundación Universitaria Los Libertadores
MACHINE LEARNING
Noviembre de 2025
Camila Andrea Hernández González

---

## Objetivo general

Desarrollar, evaluar e interpretar un clasificador reproducible para predecir diabetes usando el conjunto de datos **Diabetes Health Indicators**, siguiendo el ciclo **CRISP-DM** de extremo a extremo, aplicando técnicas de balanceo cuando sea necesario y priorizando una métrica clave con justificación técnica.

## 1. Ciclo CRISP-DM aplicado al problema

1. Business understanding

   El objetivo es identificar personas con alta probabilidad de tener diabetes a partir de variables sociodemográficas, de estilo de vida y clínicas, para apoyar acciones de prevención y diagnóstico oportuno.
   El costo de error más importante es el falso negativo: un caso de diabetes que el modelo no detecta.

2. Data understanding

   Se trabaja con un conjunto de datos sintético de salud con **100 000 observaciones** y **31 variables**, que incluyen edad, género, hábitos (actividad física, consumo de alcohol), antecedentes familiares, presión arterial, lípidos, glucosa, HbA1c, entre otros.
   La variable de respuesta es `diagnosed_diabetes` (0 = sin diagnóstico, 1 = con diagnóstico).

3. Data preparation

   - Revisión de tipos de datos y valores faltantes.
   - Separación de la variable objetivo `diagnosed_diabetes`.
   - Exclusión de variables que implican leakage: `diabetes_stage` y `diabetes_risk_score`.
   - Identificación de variables numéricas y categóricas para el preprocesamiento.

4. Modeling

   - Modelo base: **Regresión logística**.
   - Pipeline con:
     - Estandarización de variables numéricas.
     - One-Hot Encoding para variables categóricas.
   - Se utiliza `class_weight="balanced"` para compensar el desbalance aproximado 60 % casos con diabetes vs 40 % sin diabetes.

5. Evaluation

   - Validación cruzada estratificada (5 folds) sobre el conjunto de entrenamiento.
   - Métricas: recall, precision, f1, ROC-AUC, PR-AUC (average precision) y balanced accuracy.
   - Se prioriza **PR-AUC** y **recall**, dado que el objetivo principal es detectar correctamente a las personas con diabetes.

6. Deployment (simulado)

   - Script ejecutable `main.py`.
   - Notebook reproducible con todas las etapas del ciclo.
   - Archivo `requirements.txt`.
   - `README.md` explicando el proyecto.
   - Repositorio en GitHub con la trazabilidad del trabajo.


Imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    ConfusionMatrixDisplay,
    roc_auc_score,
    average_precision_score,
    RocCurveDisplay,
    PrecisionRecallDisplay,
)


Carga de datos y descriptiva básica

In [ ]:
df = pd.read_csv("diabetes_dataset.csv")

print("======================================")
print("1. ESTADÍSTICA DESCRIPTIVA BÁSICA")
print("======================================\n")

print("Shape (filas, columnas):", df.shape)

print("\nColumnas:")
print(df.columns)

print("\nPrimeras filas:")
print(df.head())

print("\nTipos de datos:")
print(df.dtypes)

print("\nValores faltantes por columna:")
print(df.isna().sum())

print("\nDistribución de 'diagnosed_diabetes' (frecuencias):")
print(df["diagnosed_diabetes"].value_counts())

print("\nDistribución de 'diagnosed_diabetes' (proporciones):")
print(df["diagnosed_diabetes"].value_counts(normalize=True))

print("\nResumen estadístico de variables numéricas:")
print(df.describe().T)


### Interpretación de la estadística descriptiva

A partir de la salida anterior se observa lo siguiente:

- El dataset tiene 100 000 observaciones y 31 columnas.
- No se identifican valores faltantes en ninguna de las variables.
- La variable objetivo `diagnosed_diabetes` está desbalanceada de manera moderada, con aproximadamente 60 % de casos positivos (1) y 40 % negativos (0).
- Las variables numéricas incluyen indicadores de estilo de vida (minutos de actividad física, horas de sueño, horas de pantalla), medidas antropométricas (IMC, relación cintura-cadera) y parámetros clínicos (presión arterial, lípidos, glucosa, HbA1c, puntaje de riesgo de diabetes).
- En general, los valores promedio de IMC, presión arterial y glucosa se encuentran en rangos compatibles con una mezcla de población sana y población con riesgo o diagnóstico de diabetes.

Esta etapa corresponde al componente de **Data Understanding** dentro de CRISP-DM.


Código (target, leakage y X / y)

In [ ]:
print("\n======================================")
print("2. DEFINICIÓN DE TARGET Y PREDICTORES")
print("======================================\n")

target = "diagnosed_diabetes"

# Variables que representan directamente el diagnóstico o riesgo y que se excluyen
cols_leakage = ["diabetes_stage", "diabetes_risk_score"]

X = df.drop(columns=[target] + cols_leakage)
y = df[target]

print("Shape de X (features):", X.shape)
print("Shape de y (target):", y.shape)


Código (train–test split)

In [ ]:
from sklearn.model_selection import train_test_split

print("\n======================================")
print("3. TRAIN-TEST SPLIT")
print("======================================\n")

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Shape X_train:", X_train.shape)
print("Shape X_test:", X_test.shape)

print("\nDistribución en y_train:")
print(y_train.value_counts(normalize=True))

print("\nDistribución en y_test:")
print(y_test.value_counts(normalize=True))


Desbalance

### Comentario sobre el desbalance de clases

El uso de `stratify=y` en el train-test split asegura que la proporción aproximada de 60 % casos con diabetes y 40 % sin diabetes se mantenga tanto en entrenamiento como en prueba.

Aunque el desbalance no es extremo, el costo de equivocarse en los casos positivos hace que se prioricen métricas sensibles a la clase minoritaria, como **recall** y **PR-AUC**, y que se utilice `class_weight="balanced"` en el modelo.

Variables numéricas y categóricas

In [ ]:
print("\n======================================")
print("4. TIPOS DE VARIABLES (CATEGÓRICAS / NUMÉRICAS)")
print("======================================\n")

cat_features = X.select_dtypes(include=["object"]).columns.tolist()
num_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

print("Variables categóricas:", cat_features)
print("\nVariables numéricas:", num_features)


Gráficos descriptivos

In [ ]:
print("\n======================================")
print("5. GRÁFICOS DESCRIPTIVOS")
print("======================================\n")

# Distribución de BMI según diagnóstico
plt.figure()
df[df["diagnosed_diabetes"] == 0]["bmi"].hist(alpha=0.6)
df[df["diagnosed_diabetes"] == 1]["bmi"].hist(alpha=0.6)
plt.xlabel("BMI")
plt.ylabel("Frecuencia")
plt.title("Distribución de BMI según diagnóstico de diabetes")
plt.legend(["No diabetes (0)", "Diabetes (1)"])
plt.tight_layout()
plt.show()

# Proporción de diabetes por género
plt.figure()
df.groupby("gender")["diagnosed_diabetes"].mean().plot(kind="bar")
plt.ylabel("Proporción con diabetes")
plt.title("Proporción de diabetes por género")
plt.tight_layout()
plt.show()


Interpretación de gráficos

### Interpretación de los gráficos descriptivos

En la distribución de IMC se observa que los valores de BMI tienden a ser más altos en el grupo con diagnóstico de diabetes, lo que es coherente con la literatura que asocia sobrepeso y obesidad con mayor riesgo de diabetes.

En el gráfico de proporción de diabetes por género se aprecian diferencias entre hombres y mujeres (según la salida concreta), lo que sugiere que el género puede aportar información útil al modelo como variable explicativa.


Pipeline y validación cruzada

In [ ]:
print("\n======================================")
print("6. PIPELINE + REGRESIÓN LOGÍSTICA")
print("======================================\n")

# Preprocesamiento: escala numéricas y codifica categóricas
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features),
    ]
)

# Modelo base: regresión logística con class_weight para compensar desbalance
log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced"
)

pipeline = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", log_reg),
    ]
)

scoring = {
    "recall": "recall",
    "precision": "precision",
    "f1": "f1",
    "roc_auc": "roc_auc",
    "pr_auc": "average_precision",
    "bal_acc": "balanced_accuracy",
}

scores = cross_validate(
    pipeline,
    X_train,
    y_train,
    cv=5,
    scoring=scoring,
    n_jobs=-1,
    return_train_score=False
)

print("Resultados de validación cruzada (5-fold):\n")
for name in scoring.keys():
    valores = scores[f"test_{name}"]
    print(f"- {name}: {valores.mean():.3f} ± {valores.std():.3f}")


Análisis de métricas de CV

### Análisis de resultados en validación cruzada

Los resultados de validación cruzada muestran:

- Valores de **recall** altos en promedio, lo que indica que el modelo identifica correctamente la mayoría de los casos con diabetes.
- La **precision** es también elevada, por lo que la mayoría de las alertas de diabetes corresponden a casos verdaderos.
- El **f1-score** resume un buen compromiso entre recall y precision.
- El **ROC-AUC** se sitúa alrededor de 0.93, lo que indica una muy buena capacidad de discriminación global entre clases.
- La **PR-AUC** (área bajo la curva Precision-Recall) es aún más alta, reflejando buen desempeño sobre la clase positiva, que es la de interés principal.
- La **balanced accuracy** supera ampliamente el 0.5, mostrando que el modelo trata de manera equilibrada ambas clases a pesar del desbalance moderado.

Con base en esto, la métrica prioritaria seleccionada es **PR-AUC**, complementada por **recall**, dado que el objetivo es detectar correctamente la mayor cantidad posible de personas con diabetes sin degradar demasiado la precisión.


Código (ajuste final, evaluación en test y gráficos)

In [ ]:
print("\n======================================")
print("7. EVALUACIÓN EN TEST")
print("======================================\n")

# Ajustar el pipeline con todos los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Predicciones
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

print("Reporte de clasificación en test:\n")
print(classification_report(y_test, y_pred))

roc = roc_auc_score(y_test, y_proba)
pr_auc = average_precision_score(y_test, y_proba)

print(f"ROC-AUC en test: {roc:.3f}")
print(f"PR-AUC (Precision-Recall AUC) en test: {pr_auc:.3f}")

# Matriz de confusión
plt.figure()
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.title("Matriz de confusión - Regresión logística")
plt.tight_layout()
plt.show()

# Curva ROC
plt.figure()
RocCurveDisplay.from_predictions(y_test, y_proba)
plt.title("Curva ROC - Regresión logística")
plt.tight_layout()
plt.show()

# Curva Precision-Recall
plt.figure()
PrecisionRecallDisplay.from_predictions(y_test, y_proba)
plt.title("Curva Precision-Recall - Regresión logística")
plt.tight_layout()
plt.show()


Conclusiones finales

## Conclusiones

El modelo de regresión logística entrenado sobre el conjunto de datos Diabetes Health Indicators, con preprocesamiento adecuado (escalado de variables numéricas y codificación de variables categóricas) y manejo del desbalance mediante `class_weight="balanced"`, logra:

- Una capacidad de discriminación alta, con **ROC-AUC** cercana a 0.93.
- Un desempeño destacado sobre la clase positiva, con **PR-AUC** en torno a 0.96–0.97.
- Valores de **recall** elevados, lo que significa que la mayoría de los pacientes con diabetes son correctamente identificados.
- Una **precision** también alta, de modo que la mayoría de los casos identificados como diabéticos efectivamente lo son.

La matriz de confusión muestra que, aunque existen falsos positivos y falsos negativos, el balance entre sensibilidad y especificidad es adecuado para un contexto donde es más costoso dejar sin detectar un caso de diabetes que generar una alerta adicional.

Este ejercicio cumple con el ciclo **CRISP-DM**:

- Business understanding: definición del problema y del costo de los errores.
- Data understanding: exploración inicial y análisis descriptivo.
- Data preparation: selección de variables y prevención de leakage.
- Modeling: construcción de un pipeline reproducible.
- Evaluation: análisis de métricas relevantes, priorizando PR-AUC y recall.
- Deployment (simulado): organización del código en `main.py`, notebook explicativo, `requirements.txt`, `README.md` y publicación en GitHub.

Como trabajo futuro, podrían explorarse modelos más complejos (árboles de decisión, Random Forest, XGBoost) y técnicas adicionales de calibración de probabilidades, así como análisis de interpretabilidad más detallados (por ejemplo, mediante SHAP).
